In [58]:
import requests
import os
import pandas as pd
import praw
import csv
import numpy as np
from datetime import datetime, timedelta
import time
import re

In [60]:
# Define crime-related keywords
crime_keywords = [
    "homicide", "murder", "killing", "manslaughter", "shooting", "shoot", "sex abuse", "rape",
    "assault", "domestic violence", "gender violence", "assault with dangerous weapon",
    "aggravated assault", "attempted", "battery", "robbery", "theft", "mugging", "beat up",
    "stealing", "steal", "robbing", "burglary", "break in", "forced entry", "housebreaking",
    "trespassing", "auto theft", "motor vehicle theft", "carjacking", "vehicle larceny",
    "arson", "destruction of property", "set on fire", "sentenced", "charges", "charged",
    "deadly", "fatal", "beating", "punching", "punched", "punches", "punch", "kicking",
    "kicked", "death", "gunshot", "gun", "gunfire", "arrested", "ammo", "ammunition",
    "shot", "abuse", "attack", "attacked", "kidnapped", "kidnap", "victim", "hate", "ransack", "stolen",
    "robbed", "gunpoint", "firearms", "carjack", "violent", "guilty", "armed", "scam"
]

In [62]:
# Function to check if a post contains any crime-related keywords
def contains_crime_keywords_in_title(post_title, crime_keywords):
    # Create a regular expression pattern that matches any of the crime keywords as whole words
    pattern = r'\b(?:' + '|'.join(map(re.escape, crime_keywords)) + r')\b'
    if re.search(pattern, post_title, re.IGNORECASE):
        return True
    return False

# Initialize Reddit instance (make sure to provide valid credentials)
reddit = praw.Reddit(
    client_id= "L9bDp7Cr1Uianzb7xtzeXw",
    client_secret= "WCJSgyUcmO9hyu-ItBO1zAKiIX-sBQ" , 
    user_agent= "python:project-script:v1.0 (by u/Username10907)" )

In [64]:


# Set the subreddit to scrape
my_subreddit = "washingtondc"
submission_headers = ['author', 'created_utc', 'id', 
                      'is_original_content', 'is_self', 
                      'link_flair_text', 'locked', 'name', 
                      'num_comments', 'over_18', 'permalink', 
                      'score', 'selftext', 'spoiler', 'stickied', 
                      'subreddit', 'title', 'upvote_ratio', 'url']

# Open CSV file to append the scraped data
with open("reddit_test_submission_db.csv", 'a', 
          encoding="utf-8", newline='') as f_object:
    newposts = reddit.subreddit(my_subreddit).new(limit=None)
    for post in newposts:
        # Check if the post title contains any crime-related keywords
        if contains_crime_keywords_in_title(post.title, crime_keywords):
            # Below are all the fields we'll request from PRAW for each post
            data = {'author': post.author, 'created_utc': post.created_utc, 'id': post.id, 
                    'is_original_content': post.is_original_content, 'is_self': post.is_self, 
                    'link_flair_text': post.link_flair_text, 'locked': post.locked, 
                    'name': post.name, 'num_comments': post.num_comments, 
                    'over_18': post.over_18, 'permalink': post.permalink, 
                    'score': post.score, 'selftext': post.selftext, 'spoiler': post.spoiler, 
                    'stickied': post.stickied, 'subreddit': post.subreddit.display_name, 
                    'title': post.title, 'upvote_ratio': post.upvote_ratio, 'url': post.url}
            
            dictwriter_object = csv.DictWriter(f_object, fieldnames=submission_headers)
            dictwriter_object.writerow(data)

# Code below will delete duplicates on successive pulls
post_db = pd.read_csv("reddit_test_submission_db.csv", names=submission_headers, header=0)
post_db.drop_duplicates(subset="permalink", keep="last", inplace=True)
post_db.to_csv("reddit_test_submission_db.csv", index=False, chunksize=1000)
